In [1]:
import tensorflow as tf
import numpy as np
import os

print("This notebook works with tensorflow version: ", tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']

for folder in folders:
  if not os.path.isdir(folder):
    os.mkdir(folder)

print(os.listdir('.'))    

This notebook works with tensorflow version:  2.2.0
['.config', 'model_name', 'weights', 'tmp', 'models', 'sample_data']


In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
#Using a model checkpoint during training
checkpoint_dir = 'weights/'

hist = model.fit(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs = 2,
    batch_size = 512, 
    callbacks = [
                 tf.keras.callbacks.ModelCheckpoint(
                     os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
                     monitor = 'val_acc', save_weights_only = True, save_best_only = True
                 )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 9ms/step - loss: 0.6954 - acc: 0.7678 - val_loss: 0.4947 - val_acc: 0.8283
Epoch 2/2
118/118 [==============================] - 1s 8ms/step - loss: 0.4425 - acc: 0.8457 - val_loss: 0.4407 - val_acc: 0.8468


In [6]:
os.listdir(checkpoint_dir)

['epoch_01_acc_0.8283.index',
 'epoch_01_acc_0.8283.data-00000-of-00001',
 'epoch_02_acc_0.8468.index',
 'checkpoint',
 'epoch_02_acc_0.8468.data-00000-of-00001']

In [7]:
#Load weights
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))



[2.3242738246917725, 0.0940999984741211]


In [9]:
model.load_weights('weights/epoch_02_acc_0.8468')
print(model.evaluate(x_test, y_test, verbose=False))

[0.44072118401527405, 0.8468000292778015]


In [10]:
#Save the entire model during training

models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 9ms/step - loss: 0.6947 - acc: 0.7629 - val_loss: 0.5026 - val_acc: 0.8267
Epoch 2/2
118/118 [==============================] - 1s 8ms/step - loss: 0.4415 - acc: 0.8449 - val_loss: 0.4520 - val_acc: 0.8437


In [11]:
os.listdir(models_dir)

['epoch_01_acc_0.8267.h5', 'epoch_02_acc_0.8437.h5']

In [13]:
#Load models
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.40872859954834, 0.07620000094175339]


In [14]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8437.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________
[0.4520431160926819, 0.8436999917030334]


In [15]:
#Manual saving of weights and models - easier to use

model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['manually_saved.data-00000-of-00001', 'checkpoint', 'manually_saved.index']


In [16]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['manually_saved.data-00000-of-00001', 'manually_saved_model.h5', 'checkpoint', 'manually_saved.index']


In [17]:
#Exporting and Restoring Saved Model Format

model.save('model_name')
print(os.listdir('model_name'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_name/assets
['saved_model.pb', 'variables', 'assets']


In [18]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose = True))

313/313 [==============================] - 0s 1ms/step - loss: 0.4520 - acc: 0.8437
[0.4520431160926819, 0.8436999917030334]
